<a href="https://colab.research.google.com/github/Braelin2/ECGR5105HW6/blob/main/ECGR5105_HW6_P1a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch as torch
import torch.optim as optim
import torch.nn as nn


from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/My Drive/ECGR5105/Housing.csv'
dataset = pd.DataFrame(pd.read_csv(file_path))
dataset.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [ ]:
#Applying a binary map
convlist =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']

# Defining the map function
def Convert(x):
    return x.map({'yes': 1, 'no': 0, 'furnished':  1, 'semi-furnished':  0, 'unfurnished':  -1})

dataset[convlist] = dataset[convlist].apply(Convert)
dataset.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,1
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,1
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,0
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,1
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,1


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

varlist =  ['area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'parking', 'prefarea', 'furnishingstatus']

training, testing = train_test_split(dataset, train_size = 0.8, test_size = 0.2, random_state = 0)

Y_train = training.values[:,0]
Y_test = testing.values[:,0]

Y_train_tensor = torch.tensor(Y_train, dtype =torch.float)
Y_test_tensor = torch.tensor(Y_test, dtype =torch.float)

standScaler = StandardScaler()

X_train_temp = training
X_train_temp[varlist] = standScaler.fit_transform(training[varlist])

X_test_temp = testing
X_test_temp[varlist] = standScaler.fit_transform(testing[varlist])

num_train = len(Y_train)
num_test = len(Y_test)

X_train_temp1 = np.c_[np.ones((num_train, 1)), X_train_temp[varlist]]
X_test_temp1 = np.c_[np.ones((num_test, 1)), X_test_temp[varlist]]

X_train_tensor = torch.tensor(X_train_temp1, dtype =torch.float)
X_test_tensor = torch.tensor(X_test_temp1, dtype =torch.float)

In [ ]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_in_train, t_in_test, t_out_train, t_out_test):
  for epoch in range(1, n_epochs + 1):
    t_p_train = model(t_in_train)
    train_loss = loss_fn(t_p_train, t_out_train)

    t_p_test = model(t_in_test)
    test_loss = loss_fn(t_p_test, t_out_test)

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    if epoch == 1 or epoch % 1000 == 0:
      print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"f" Validation loss {test_loss.item():.4f}")

In [ ]:
seq_model = nn.Sequential(nn.Linear(13, 32), nn.Tanh(), nn.Linear(32, 1))

optimize = optim.SGD(seq_model.parameters(),lr=1e-3)

training_loop(n_epochs = 5000,
              optimizer=optimize,
              model = seq_model,
              loss_fn = nn.MSELoss(),
              t_in_train = X_train_tensor,
              t_in_test = X_test_tensor,
              t_out_train = Y_train_tensor,
              t_out_test = Y_test_tensor)


Epoch 1, Training loss 26469977817088.0000, Validation loss 25189242896384.0000
Epoch 1000, Training loss 3644181970944.0000, Validation loss 2884937449472.0000
Epoch 2000, Training loss 3644181970944.0000, Validation loss 2884937449472.0000
Epoch 3000, Training loss 3644181970944.0000, Validation loss 2884937449472.0000
Epoch 4000, Training loss 3644181970944.0000, Validation loss 2884937449472.0000
Epoch 5000, Training loss 3644181970944.0000, Validation loss 2884937449472.0000


In [ ]:
print(sum([p.numel() for p in seq_model.parameters()]))

481
